## Code For Frozen Bert Weights for Regression
- This model trains and tests solely on patients who did not die

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from multimodal import MultimodalDataset, MultimodalNetwork, collation
from sklearn.preprocessing import StandardScaler
from transformers import AutoModel
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [2]:
!nvidia-smi

Fri Apr  5 15:27:52 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   63C    P2   297W / 300W |   8503MiB / 49140MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
base_path = 'data/regression/partitioned'

static_train = pd.read_csv(f'{base_path}/static_alive_train.csv')
static_val = pd.read_csv(f'{base_path}/static_alive_val.csv')
static_test = pd.read_csv(f'{base_path}/static_alive_test.csv')

In [4]:
notes = pd.read_csv('data/notes_with_interval.csv')
notes_train = notes[notes['id'].isin(static_train['id'])]
notes_val = notes[notes['id'].isin(static_val['id'])]
notes_test = notes[notes['id'].isin(static_test['id'])]

In [5]:
dynamic = pd.read_csv('data/dynamic_cleaned.csv')
dynamic_train = dynamic[dynamic['id'].isin(static_train['id'])].copy()
dynamic_val = dynamic[dynamic['id'].isin(static_val['id'])].copy()
dynamic_test = dynamic[dynamic['id'].isin(static_test['id'])].copy()

In [6]:
features = ['aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium']

scaler = StandardScaler()

dynamic_train.loc[:, features] = scaler.fit_transform(dynamic_train[features])
dynamic_val.loc[:, features] = scaler.transform(dynamic_val[features])
dynamic_test.loc[:, features] = scaler.transform(dynamic_test[features])  

dynamic_train.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium
0,26115624,9/7/50 0:22,-0.717898,-0.348539,-1.001482,-0.658472,1.443217,-0.739575,-0.574781,0.738555,-0.905187
3,28478629,10/8/96 5:30,0.438966,0.602864,1.888824,-1.191739,-0.700705,0.650850,-0.020353,-0.360615,0.692710
5,28478629,10/13/96 4:35,0.207593,2.315391,1.851287,-0.658472,-1.844130,0.032883,-0.747269,-0.360615,0.402183
7,22576776,1/13/84 7:30,1.364456,-0.538820,-1.114091,-0.125205,0.156864,-0.636581,-0.574781,0.555360,-0.614660
8,22576776,1/13/84 7:30,1.364456,-0.538820,-1.114091,-0.125205,0.156864,-0.636581,-0.574781,0.555360,-0.614660


### Dynamic train cleaning

In [7]:
id_lengths_train = dynamic_train['id'].value_counts().to_dict()
dynamic_train = dynamic_train.sort_values(by=['id', 'charttime'])
dynamic_train = dynamic_train.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_train['id']).agg(list)

dynamic_train

id
20008098    [[0.6703382238753892, -0.9193815984002599, -0....
20013244    [[-0.48652535007205266, 0.6028643994201031, -0...
20021110    [[0.20759279429641245, 0.6028643994201031, -0....
20024177    [[0.6703382238753892, 0.032022150237466976, -0...
20025344    [[-0.023779920493075934, 0.6028643994201031, 2...
                                  ...                        
29988601    [[0.9017109386648776, -0.34853934921762375, 0....
29990494    [[0.9017109386648776, -0.7291008486727145, -0....
29992506    [[-0.7178980648615411, 0.41258364969255773, -0...
29994296    [[-0.48652535007205266, -0.9193815984002599, 1...
29997500    [[-0.7178980648615411, 2.6959526464231023, -0....
Length: 2143, dtype: object

### Dynamic val cleaning

In [8]:
id_lengths_val = dynamic_val['id'].value_counts().to_dict()
dynamic_val = dynamic_val.sort_values(by=['id', 'charttime'])
dynamic_val = dynamic_val.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_val['id']).agg(list)

dynamic_val

id
20015730    [[-0.9492707796510295, 0.22230289996501235, -0...
20020562    [[-0.9492707796510295, 0.032022150237466976, -...
20024788    [[1.3644563682438544, -0.9193815984002599, 1.7...
20076965    [[2.7526926569807846, -0.34853934921762375, 1....
20226142    [[-0.48652535007205266, 1.1737066486027392, -0...
                                  ...                        
29760041    [[0.6703382238753892, -1.1096623481278052, -0....
29794789    [[0.6703382238753892, -1.2999430978553506, 0.1...
29887340    [[1.3644563682438544, -1.2999430978553506, 0.2...
29895525    [[1.133083653454366, 0.6028643994201031, 1.363...
29923625    [[-0.48652535007205266, 0.6028643994201031, -0...
Length: 239, dtype: object

### Dynamic test cleaning

In [9]:
id_lengths_test = dynamic_test['id'].value_counts().to_dict()
dynamic_test = dynamic_test.sort_values(by=['id', 'charttime'])
dynamic_test = dynamic_test.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_test['id']).agg(list)

dynamic_test

id
20089631    [[0.9017109386648776, -0.34853934921762375, 1....
20100253    [[-0.9492707796510295, 0.032022150237466976, 0...
20111879    [[-0.9492707796510295, 0.22230289996501235, 0....
20136761    [[-0.9492707796510295, 0.22230289996501235, -0...
20176432    [[-1.4120162092300061, 1.1737066486027392, -0....
                                  ...                        
29871673    [[1.133083653454366, 0.22230289996501235, -0.7...
29921759    [[-0.9492707796510295, 0.7931451491476484, -0....
29970938    [[-0.2551526352825643, -0.1582585994900784, -0...
29981653    [[-0.48652535007205266, 0.41258364969255773, -...
29990184    [[-0.023779920493075934, 1.9348296475129207, 0...
Length: 265, dtype: object

In [10]:
notes = notes[['id', 'charttime', 'text', 'interval']]

notes_train = notes[notes['id'].isin(static_train['id'])].copy()
notes_val = notes[notes['id'].isin(static_val['id'])].copy()
notes_test = notes[notes['id'].isin(static_test['id'])].copy()

In [11]:
train_data = MultimodalDataset(static=static_train, dynamic=dynamic_train, id_lengths=id_lengths_train, notes=notes_train)
validation_data = MultimodalDataset(static=static_val, dynamic=dynamic_val, id_lengths=id_lengths_val, notes=notes_val)

train_loader = DataLoader(train_data, batch_size=1500, shuffle=True, collate_fn=collation)
val_loader = DataLoader(validation_data, batch_size=150, shuffle=True, collate_fn=collation)

In [12]:
text_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

for params in text_model.parameters():
    params.requires_grad = False

model = MultimodalNetwork(input_size=9, out_features=1, hidden_size=128, text_model=text_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f'device: {device}')

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device: cuda


In [13]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
    
    def forward(self, predicted, actual):
        return torch.sqrt(self.mse(predicted, actual))

In [14]:
criterion = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
!nvidia-smi

Fri Apr  5 15:28:12 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 39%   71C    P2   297W / 300W |  10143MiB / 49140MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
loss_base_path = 'data/losses/alive_regression'

In [17]:
epochs = 200
training_loss = []
validation_loss = []
patience = 10
stagnation = 0

for epoch in range(1, epochs+1):
    print(f'training epoch: [{epoch}/{epochs}]')
    model.train()
    training_loss_epoch = 0

    for step, batch in enumerate(train_loader):
        packed_dynamic_X, notes_X, notes_intervals, los = batch

        packed_dynamic_X = packed_dynamic_X.to(device)
        los = los.to(device)

        notes_X_gpu = []
        for notes in notes_X:
            notes_gpu = {key: value.to(device) for key, value in notes.items()}
            notes_X_gpu.append(notes_gpu)

        outputs = model(packed_dynamic_X, notes_X_gpu, notes_intervals)

        loss = criterion(outputs, los)
        training_loss_epoch += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % max(1, round(len(train_loader) * 0.1)) == 0:
            print(f'step: [{step+1}/{len(train_loader)}] | loss: {loss.item():.4}')

            if step+1 == 1 and epoch == 1:
                with open(f'{loss_base_path}/loss_step.txt', 'w') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

            else:
                with open(f'{loss_base_path}/loss_step.txt', 'a') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

    avg_training_loss_epoch = training_loss_epoch / len(train_loader)
    training_loss.append(avg_training_loss_epoch.item())
    print(f'Training epoch loss: {avg_training_loss_epoch.item():.4f}\n')

    if epoch == 1:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'w') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    else:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'a') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    print(f'validation epoch: [{epoch}/{epochs}]')
    
    model.eval()
    with torch.no_grad():
        validation_loss_epoch = 0

        for val_step, val_batch in enumerate(val_loader):
            packed_dynamic_X_val, notes_X_val, notes_intervals_val, los_val = val_batch

            packed_dynamic_X_val = packed_dynamic_X_val.to(device)
            los_val = los_val.to(device)

            notes_X_val_gpu = []
            for notes in notes_X_val:
                notes_val_gpu = {key: value.to(device) for key, value in notes.items()}
                notes_X_val_gpu.append(notes_val_gpu)

            val_outputs = model(packed_dynamic_X_val, notes_X_val_gpu, notes_intervals_val)
            val_loss = criterion(val_outputs, los_val)
            validation_loss_epoch += val_loss

        avg_validation_loss = validation_loss_epoch / len(val_loader)
        print(f'Validation epoch loss: {avg_validation_loss.item():.4f}\n')
        
        if len(validation_loss) == 0 or (avg_validation_loss.item() < min(validation_loss)):
            stagnation = 0
            torch.save(model.state_dict(), 'data/models/alive_regression_best_model.pth')
            print(f'new minimum validation loss')
            print(f'model saved\n')

        else:
            stagnation += 1

        validation_loss.append(avg_validation_loss.item())

        if epoch == 1:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'w') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        else:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'a') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        if stagnation >= patience:
            print(f'No improvement over {patience} epochs')
            print('Early stopping')
            break

    model.train()

    print('===============================\n')
    

training epoch: [1/200]
step: [1/2] | loss: 7.812
step: [2/2] | loss: 6.73
Training epoch loss: 7.2711

validation epoch: [1/200]
Validation epoch loss: 8.1562

new minimum validation loss
model saved


training epoch: [2/200]
step: [1/2] | loss: 6.569
step: [2/2] | loss: 7.882
Training epoch loss: 7.2257

validation epoch: [2/200]
Validation epoch loss: 7.5642

new minimum validation loss
model saved


training epoch: [3/200]
step: [1/2] | loss: 6.492
step: [2/2] | loss: 6.626
Training epoch loss: 6.5590

validation epoch: [3/200]
Validation epoch loss: 7.1678

new minimum validation loss
model saved


training epoch: [4/200]
step: [1/2] | loss: 6.589
step: [2/2] | loss: 5.463
Training epoch loss: 6.0261

validation epoch: [4/200]
Validation epoch loss: 7.0253

new minimum validation loss
model saved


training epoch: [5/200]
step: [1/2] | loss: 6.415
step: [2/2] | loss: 5.755
Training epoch loss: 6.0851

validation epoch: [5/200]
Validation epoch loss: 7.2103


training epoch: [6/200